In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import math
import time
import os

In [95]:
city_url = 'https://geographic.org/streetview/usa/ca/orange/index.html'
response = requests.get(city_url)
soup = BeautifulSoup(response.text, 'html.parser')
cities = soup.body.ul.find_all('li')
clean_cities = []
for city in cities:
    clean_cities.append(f'{city.text}, CA')
print(clean_cities)

In [240]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = f'https://www.redfin.com/city/9361/CA/Irvine/filter/property-type=house,status=active'
browser.visit(url)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\adams\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [105]:
state = 'CA'
for city in clean_cities:
    browser.fill('searchInputBox',f'{city}\n')
    time.sleep(4) #Delay is necessary but time can be adjusted
    try:
        browser.find_by_id('download-and-save').click()
    except:
        try:
            browser.find_by_xpath('/html/body/div[4]/div/div[2]/div/div/div/div[2]/div/div/div[2]/div[1]/div/div[2]').click()
            browser.find_by_id('download-and-save').click()
        except:
            continue
    browser.fill('searchInputBox','')
    browser.find_by_xpath('/html/body/div[1]/div[3]/div/div/header[2]/div[1]/div[2]/div/form/div[1]/div/div/div/div/input').click()



In [106]:
!mv ~/Downloads/redfin* ~/gitTemp/FinalProject/Data/Dirty

In [127]:
path_to_data = './Data'
file_list = os.listdir(path_to_data)
for file in file_list:
    temp_csv = pd.read_csv(f'{path_to_data}/{file}')
    temp_df = pd.DataFrame(temp_csv)
    try:
        redfin_df = redfin_df.append(temp_df)
    except:
        redfin_df = temp_df
redfin_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [141]:
#Only take houses currently listed
redfin_df = redfin_df[redfin_df['SALE TYPE'] == 'MLS Listing']

Single Family Residential    3868
Name: PROPERTY TYPE, dtype: int64

In [142]:
del redfin_df['SOLD DATE']
del redfin_df['SALE TYPE']
del redfin_df['NEXT OPEN HOUSE END TIME']
del redfin_df['NEXT OPEN HOUSE START TIME']
del redfin_df['SOURCE']
del redfin_df['FAVORITE']
del redfin_df['INTERESTED']
del redfin_df['STATUS']
del redfin_df['HOA/MONTH']
del redfin_df['PROPERTY TYPE']
#Location is unnecessary as street, city, state, and zipcode are all included in other columns 
del redfin_df['LOCATION']

In [154]:
#Remove redundant listings
redfin_df = redfin_df.drop_duplicates()
redfin_df.reset_index(inplace=True)
del redfin_df['index']

In [276]:
listing_url = redfin_df['URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)']
house_url = []
school_name = []
school_rating = []
school_type = []
school_grades = []
average_rating = []

In [283]:
#They don't like webscraping so I have to manually resume scraping when it hits a recaptcha
begin = 637

for i in range(begin, len(listing_url)):
    rating = 0
    browser.visit(listing_url[i])
    soup = BeautifulSoup(browser.html, 'html.parser')
    try:
        school_info = soup.find('div', class_='schools-content').table.find_all('tr')
        for school in school_info:
            house_url.append(listing_url[i])
            try:
                school_name.append(school.find('div', class_='school-name').text)
            except:
                school_name.append('issue')
            try:
                school_rating.append(float(school.find("span", class_="rating-num").text))
            except:
                school_rating.append('issue')
            try:
                rating += float(school.find("span", class_="rating-num").text)
            except:
                rating = 'issue'
            try:
                school_serves = school.find('div', class_='sub-info').text.split(' • ')
                try:
                    school_type.append(school_serves[0])
                except:
                    school_type.append('issue')
                try:
                    school_grades.append(school_serves[1])
                except:
                    school_grades.append('issue')
            except:
                school_type.append('issue')
                school_grades.append('issue')
        try:
            average_rating.append(round(rating / len(school_info), 3))
        except:
            average_rating.append('issue')
    except:
        try:
            ghost_town = soup.find('div', class_='GhostTown').text
            if ghost_town == "We couldn't find any schools that service this home or are within 5 miles of this home.":
                average_rating.append('No Schools')
        except:
            break
    print(i)

637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886


In [287]:
redfin_df['Average School Rating'] = average_rating

<ipython-input-287-a85e63c3b8ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redfin_df['Average School Rating'] = average_rating


In [300]:
clean_redfin = redfin_df[redfin_df['Average School Rating'] != 'issue']
clean_redfin.reset_index(inplace = True)
del clean_redfin['index']

In [296]:
school_df = pd.DataFrame({
    'Listing URL': house_url,
    'School Name': school_name,
    'School Rating': school_rating,
    'School Type': school_type,
    'School Grades': school_grades
})
school_df.to_csv('./Data/School_Data.csv')

In [295]:
#Probably need to drop the 605 columns with School Rating issues but left for now
school_df[school_df['School Rating'] != 'issue']

,Listing URL,School Name,School Rating,School Type,School Grades
0,http://www.redfin.com/CA/Aliso-Viejo/11-Hazelb...,Don Juan Avila Elementary,8,Public,K to 5
1,http://www.redfin.com/CA/Aliso-Viejo/11-Hazelb...,Don Juan Avila Middle,8,Public,6 to 8
2,http://www.redfin.com/CA/Aliso-Viejo/11-Hazelb...,Aliso Niguel High,9,Public,9 to 12
3,http://www.redfin.com/CA/Laguna-Beach/11-Sande...,El Morro Elementary,9,Public,K to 5
4,http://www.redfin.com/CA/Laguna-Beach/11-Sande...,Thurston Middle,9,Public,6 to 8
...,...,...,...,...,...
5606,http://www.redfin.com/CA/Yorba-Linda/4115-Live...,Bernardo Yorba Middle,8,Public,7 to 8
5607,http://www.redfin.com/CA/Yorba-Linda/4115-Live...,Yorba Linda High,10,Public,9 to 12
5608,http://www.redfin.com/CA/Yorba-Linda/20360-Flo...,Mabel M. Paine Elementary,8,Public,K to 5
5609,http://www.redfin.com/CA/Yorba-Linda/20360-Flo...,Yorba Linda Middle,9,Public,6 to 8


In [27]:
clean_redfin.reset_index(inplace=True)
del clean_redfin['index']
clean_redfin

,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),MLS#,LATITUDE,LONGITUDE,Average School Rating
0,11 Hazelbranch,Aliso Viejo,CA,92656,975000,3,2.50,1470.0,4950.0,1993.0,3.0,663.0,http://www.redfin.com/CA/Aliso-Viejo/11-Hazelb...,OC21181915,33.592613,-117.738368,8.333
1,11 Sanderling Ln,Aliso Viejo,CA,92656,850000,3,2.50,1393.0,3000.0,1989.0,3.0,610.0,http://www.redfin.com/CA/Laguna-Beach/11-Sande...,LG21181169,33.592240,-117.749226,9.0
2,5 Skygate,Aliso Viejo,CA,92656,1350000,4,3.00,2918.0,5040.0,1989.0,5.0,463.0,http://www.redfin.com/CA/Aliso-Viejo/5-Skygate...,OC21160272,33.594179,-117.740063,8.333
3,15 Starling Ln,Aliso Viejo,CA,92656,995000,3,2.50,1605.0,3480.0,1989.0,6.0,620.0,http://www.redfin.com/CA/Aliso-Viejo/15-Starli...,LG21176453,33.596696,-117.744324,9.0
4,78 Summerland Cir,Aliso Viejo,CA,92656,1549000,4,4.50,3258.0,5546.0,2011.0,10.0,475.0,http://www.redfin.com/CA/Aliso-Viejo/78-Summer...,CV21176676,33.595652,-117.723661,7.667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,5191 Wendover Rd,Yorba Linda,CA,92886,1974000,7,5.25,3463.0,18270.0,1965.0,87.0,570.0,http://www.redfin.com/CA/Yorba-Linda/5191-Wend...,PW21107820,33.885782,-117.797048,8.333
1335,5461 Mountain View Ave,Yorba Linda,CA,92886,1087000,4,3.00,1803.0,9600.0,1961.0,99.0,603.0,http://www.redfin.com/CA/Yorba-Linda/5461-Moun...,OC21104129,33.881948,-117.804192,8.667
1336,18950 Suncrest Dr,Yorba Linda,CA,92886,3250000,5,5.50,5364.0,23681.0,2002.0,104.0,606.0,http://www.redfin.com/CA/Yorba-Linda/18950-Sun...,OC21099744,33.890639,-117.803505,9.333
1337,4115 Live Oak Ln,Yorba Linda,CA,92886,4799999,6,7.50,9302.0,43560.0,1991.0,125.0,516.0,http://www.redfin.com/CA/Yorba-Linda/4115-Live...,OC21082623,33.901382,-117.771741,9.0


In [22]:
clean_redfin = clean_redfin.dropna()

In [14]:
for col in clean_redfin:
    print(col)
    print(clean_redfin[col].max())
    print(clean_redfin[col].min())
    print()

ADDRESS
998 Emerald Bay
0 Hot Springs Cy #7

CITY
Yorba Linda
Aliso Viejo

STATE OR PROVINCE
LA
CA

ZIP OR POSTAL CODE
92887
90620

PRICE
65000000
210000

BEDS
62
1

BATHS
20.5
1.0

SQUARE FEET
29604.0
480.0

LOT SIZE
279655200.0
10.0

YEAR BUILT
2022.0
1890.0

DAYS ON MARKET
919.0
1.0

$/SQUARE FEET
20630.0
100.0

URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)
http://www.redfin.com/CA/Yorba-Linda/6652-Vista-Loma-92886/home/17477109
http://www.redfin.com/CA/Aliso-Viejo/11-Emerald-Ter-92656/home/17472131

MLS#
WS21184769
200007349

LATITUDE
33.9470271
33.390491

LONGITUDE
-117.5633653
-118.1121878

Average School Rating
No Schools
2.333



In [28]:
clean_redfin.to_csv('./Data/Clean_Real_Estate.csv')

In [3]:
clean_redfin = pd.read_csv('./Data/Clean_Real_Estate.csv')
del clean_redfin['Unnamed: 0']

In [33]:
clean_redfin['CITY'].unique()

array(['Aliso Viejo', 'Laguna Hills', 'Buena Park', 'Anaheim Hills',
       'Anaheim', 'Garden Grove', 'Orange', 'Yorba Linda', 'Placentia',
       'Fullerton', 'Brea', 'Cypress', 'La Mirada', 'Dana Point',
       'San Clemente', 'Newport Beach', 'Costa Mesa', 'Huntington Beach',
       'Stanton', 'Los Alamitos', 'Laguna Beach', 'Laguna Niguel',
       'Foothill Ranch', 'Fountain Valley', 'Westminster', 'Sunset Beach',
       'Seal Beach', 'Surfside', 'Irvine', 'Tustin', 'Lake Forest',
       'La Habra', 'La Habra Heights', 'La Palma', 'Ladera Ranch',
       'San Juan Capistrano', 'Rossmoor', 'Midway City', 'Mission Viejo',
       'Trabuco Canyon', 'Newport Coast', 'Villa Park', 'Santa Ana',
       'North Tustin', 'Rancho Santa Margarita', 'Coto de Caza',
       'SAN CLEMENTE'], dtype=object)

In [4]:
OC_Crime = pd.read_csv('./Data/OC_crime.csv')

In [5]:
crime_cities = list(set(OC_Crime['City']).intersection(set(clean_redfin['CITY'].unique())))

In [6]:
OC_Crime['City']

0                    Anaheim
1                       Brea
2                 Buena Park
3                 Costa Mesa
4                    Cypress
5                   La Palma
6            Fountain Valley
7                  Fullerton
8               Garden Grove
9           Huntington Beach
10              Laguna Beach
11                  La Habra
12              Los Alamitos
13             Newport Beach
14                    Orange
15                 Placentia
16              San Clemente
17       San Juan Capistrano
18                 Santa Ana
19                Seal Beach
20                   Stanton
21                    Tustin
22                Villa Park
23               Westminster
24               Yorba Linda
25                    Irvine
26             Mission Viejo
27                Dana Point
28             Laguna Niguel
29              Laguna Hills
30               Lake Forest
31              Laguna Woods
32    Rancho Santa Margarita
33               Aliso Viejo
Name: City, dt

In [7]:
new_redfin = clean_redfin[clean_redfin['CITY'].isin(crime_cities)]

In [8]:
OC_pop = pd.read_csv('./Data/Dirty/OC_population.csv')

In [16]:
OC_pop = OC_pop.dropna()
OC_pop = OC_pop[OC_pop['Name'].isin(crime_cities)]
OC_pop.reset_index(inplace = True)
del OC_pop['index']

,Name,Status,County,Population,Population.1,Population.2,Population.3
0,Aliso Viejo,City,Orange,"6,605","39,194","47,668","52,176"
1,Anaheim,City,Orange,"266,229","327,920","336,332","346,824"
2,Brea,City,Orange,"33,193","35,556","39,195","47,325"
3,Buena Park,City,Orange,"68,594","78,456","80,456","84,034"
4,Costa Mesa,City,Orange,"96,354","109,371","110,082","111,918"
5,Cypress,City,Orange,"42,634","46,453","47,858","50,151"
6,Dana Point,City,Orange,"31,781","35,119","33,291","33,107"
7,Fountain Valley,City,Orange,"54,204","54,921","55,360","57,047"
8,Fullerton,City,Orange,"114,235","126,187","135,183","143,617"
9,Garden Grove,City,Orange,"144,216","165,242","170,960","171,949"


In [25]:
clean_OC_pop = OC_pop[['Name','Population.3']]
clean_OC_pop = clean_OC_pop.rename(columns = {'Name': 'City','Population.3':'Population'})

In [43]:
crime_per_cap = clean_OC_pop.merge(OC_Crime, on='City', how='inner')
crime_per_cap = crime_per_cap[['City','Population','Offences_Total']]
pops = []
for pop in crime_per_cap['Population']:
    pop = pop.replace(',','')
    pops.append(int(pop))
crime_per_cap['Population'] = pops

In [44]:
crime_per_cap['Crime per Capita (1000s)'] = (crime_per_cap['Offences_Total'] / crime_per_cap['Population']) *1000

In [61]:
crime_list = []
for city in new_redfin['CITY']:
    crime_list.append(float(crime_per_cap[crime_per_cap['City'] == city]['Crime per Capita (1000s)']))
new_redfin['Crime per Capita (1000s)'] = crime_list

<ipython-input-61-96dbff0273cd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_redfin['Crime per Capita (1000s)'] = crime_list


In [65]:
new_redfin.reset_index(inplace=True)
del new_redfin['index']
new_redfin

,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),MLS#,LATITUDE,LONGITUDE,Average School Rating,Crime per Capita (1000s)
0,11 Hazelbranch,Aliso Viejo,CA,92656,975000,3,2.50,1470.0,4950.0,1993.0,3.0,663.0,http://www.redfin.com/CA/Aliso-Viejo/11-Hazelb...,OC21181915,33.592613,-117.738368,8.333,1.629102
1,11 Sanderling Ln,Aliso Viejo,CA,92656,850000,3,2.50,1393.0,3000.0,1989.0,3.0,610.0,http://www.redfin.com/CA/Laguna-Beach/11-Sande...,LG21181169,33.592240,-117.749226,9.0,1.629102
2,5 Skygate,Aliso Viejo,CA,92656,1350000,4,3.00,2918.0,5040.0,1989.0,5.0,463.0,http://www.redfin.com/CA/Aliso-Viejo/5-Skygate...,OC21160272,33.594179,-117.740063,8.333,1.629102
3,15 Starling Ln,Aliso Viejo,CA,92656,995000,3,2.50,1605.0,3480.0,1989.0,6.0,620.0,http://www.redfin.com/CA/Aliso-Viejo/15-Starli...,LG21176453,33.596696,-117.744324,9.0,1.629102
4,78 Summerland Cir,Aliso Viejo,CA,92656,1549000,4,4.50,3258.0,5546.0,2011.0,10.0,475.0,http://www.redfin.com/CA/Aliso-Viejo/78-Summer...,CV21176676,33.595652,-117.723661,7.667,1.629102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,5191 Wendover Rd,Yorba Linda,CA,92886,1974000,7,5.25,3463.0,18270.0,1965.0,87.0,570.0,http://www.redfin.com/CA/Yorba-Linda/5191-Wend...,PW21107820,33.885782,-117.797048,8.333,0.834114
1260,5461 Mountain View Ave,Yorba Linda,CA,92886,1087000,4,3.00,1803.0,9600.0,1961.0,99.0,603.0,http://www.redfin.com/CA/Yorba-Linda/5461-Moun...,OC21104129,33.881948,-117.804192,8.667,0.834114
1261,18950 Suncrest Dr,Yorba Linda,CA,92886,3250000,5,5.50,5364.0,23681.0,2002.0,104.0,606.0,http://www.redfin.com/CA/Yorba-Linda/18950-Sun...,OC21099744,33.890639,-117.803505,9.333,0.834114
1262,4115 Live Oak Ln,Yorba Linda,CA,92886,4799999,6,7.50,9302.0,43560.0,1991.0,125.0,516.0,http://www.redfin.com/CA/Yorba-Linda/4115-Live...,OC21082623,33.901382,-117.771741,9.0,0.834114


In [66]:
new_redfin.to_csv('./Data/Clean_Real_Estate_With_Crime.csv')